In [1]:
include("./Bilevel/Bilevel.jl")
include("utils.jl")
using .Bilevel
using Plots
using ArgParse, PrettyTables

In [ ]:
# -- DEFAULT PARAMETERS OF A SIMULATION --
verbose     = true

# ------- Choice of the model -------
bilevel     = false 
storage     = false 

# ------- Profiles parameters -------
nb_days     = 2 
delta_t     = 60 
peak_power  = 7.0 
EV          = false 
EHP         = false 

# ------- Users parameters -------
PV_pen        = 1.0     
storage_pen   = 1.0   
storage_eff   = 0.98
storage_cost  = 500.0   # kEUR/MVA
amort_storage = 15      # years
PV_capa       = 0.4     # max in MVA
PV_cost       = 500.0   # EUR/kWc 
PV_conv_cost  = 200.0   # kEUR/MVA
amort_PV_conv = 10      # years
amort_PV      = 25      # years 
EIC           = 0.3     # EUR/kWh  
EEC           = 0.1     # EUR/kWh   
DSOEC         = 0.1     # EUR/kWh  
GCC           = 80.0    # EUR/kVA_year
cos_phi       = 0.95  

# ------- DSO parameters -------
substation_cost   = 1e3  # kEUR/MVA
amort_DSO         = 50  
interest_rate_DSO = 0.06 
money_basis = 1.0
weight_I = weight_V = 1.0e-2
weight_obj1 = weight_obj2 = 1.0

In [2]:
include("main_bilevel.jl")
network.buses

#--------------------------------------------------------------------------------
#                                                                                
#                                  Bilevel DNEP                                   
#                                                                                
#                                Graduation work                                 
#                                                                                
#--------------------------------------------------------------------------------
# @ Manon Cornet

Running a simulation with the following characteristics:
--------------------------------------------------------

╭─────────────┬───────┬───────┬─────────┬─────────┬─────────┬─────────┬─────────┬───────────╮
│  Model Type │    EV │   EHP │ Delta_t │ PV_CAPA │     EIC │     EEC │   DSOEC │       GCC │
│         [-] │   [-] │   [-] │   [min] │   [MVA] │ [€/kWh] │ [€/kWh] │ [€/kWh] │ [€/kVA/y] │
├─────────

4-element Vector{Main.Bilevel.Bus}:
 Main.Bilevel.Substation(Main.Bilevel.Node(1, (x = 0.0, y = 0.1)), false, (V_min = 0.95, V_max = 1.05), 15.0, 0.0, nothing, nothing, nothing)
 Main.Bilevel.Substation(Main.Bilevel.Node(2, (x = 0.0, y = 0.0)), false, (V_min = 0.95, V_max = 1.05), 15.0, 0.0, nothing, nothing, nothing)
 Main.Bilevel.User(Main.Bilevel.Node(3, (x = 0.1, y = 0.1)), (V_min = 0.95, V_max = 1.05), 0.4, nothing, Main.Bilevel.Profile([0.86704535386329, 0.6751924683412124, 0.24981686633378278, 1.5789764924404954], 720), Main.Bilevel.PV(Main.Bilevel.Profile([1.0, 0.9190765189761652, 0.1, 0.09190765189761652], 720), (max_q = 0.3, slope = -1.0), nothing, nothing, nothing), nothing, 0.95)
 Main.Bilevel.User(Main.Bilevel.Node(4, (x = 0.2, y = 0.1)), (V_min = 0.95, V_max = 1.05), 0.4, nothing, Main.Bilevel.Profile([0.28666628695107366, 0.24837434261737962, 0.46309441311790805, 0.7650165650927456], 720), nothing, nothing, 0.95)

In [3]:
model = Bilevel.build_model(simulation; set_names=true)
simulation.storage

Set parameter Username
Academic license - for non-commercial use only - expires 2023-11-10
Set parameter TimeLimit to value 600
Set parameter MIPGap to value 0.01
Set parameter MIPFocus to value 1


[ Info: Building model...
[ Info: Built model in 6.20 seconds


false

In [4]:
return_value = Bilevel.solve_model(model, formulation.powerflow)
save_struct(network_topology, "network_topology.json")
save_struct(network, "network_data.json")

Set parameter MIPGap to value 0.01
Set parameter MIPFocus to value 1
Set parameter TimeLimit to value 600
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[arm])

CPU model: Apple M1 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 472 rows, 313 columns and 1768 nonzeros
Model fingerprint: 0x8f0df180
Model has 20 quadratic constraints
Variable types: 296 continuous, 17 integer (17 binary)
Coefficient statistics:
  Matrix range     [1e-09, 2e+05]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [2e-02, 1e+00]
  Bounds range     [4e-01, 1e+00]
  RHS range        [8e-02, 1e+01]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 202 rows and 75 columns
Presolve time: 0.00s
Presolved: 270 rows, 238 columns, 1194 nonzeros
Presolved model has 20 quadratic constraint(s)
Variable types: 224 continuous, 14 integer (14 binary)

Root relaxation: objective 3.8

In [27]:
thesis_dir = "/Users/manoncornet/Documents/University/TFE/ThesisWriting/Master_Thesis/figures" 
for t in 1:model[:time_steps]
    Bilevel.print_network_tikz(model[:network_data], t, 40, 40; dir=thesis_dir, filename="OneLevel_graph_$(t)")
end

DaisyDaisyThis is LuaHBTeX, Version 1.15.0 (TeX Live 2022) 
 restricted system commands enabled.
(./OneLevel_graph_1.tex
LaTeX2e <2021-11-15> patch level 1
 L3 programming layer <2022-02-24>
(/usr/local/texlive/2022/texmf-dist/tex/latex/standalone/standalone.cls
Document Class: standalone 2018/03/26 v1.3a Class to compile TeX sub-files stan
dalone
(/usr/local/texlive/2022/texmf-dist/tex/latex/tools/shellesc.sty)
(/usr/local/texlive/2022/texmf-dist/tex/generic/iftex/ifluatex.sty
(/usr/local/texlive/2022/texmf-dist/tex/generic/iftex/iftex.sty))
(/usr/local/texlive/2022/texmf-dist/tex/latex/xkeyval/xkeyval.sty
(/usr/local/texlive/2022/texmf-dist/tex/generic/xkeyval/xkeyval.tex
(/usr/local/texlive/2022/texmf-dist/tex/generic/xkeyval/xkvutils.tex
(/usr/local/texlive/2022/texmf-dist/tex/generic/xkeyval/keyval.tex))))
(/usr/local/texlive/2022/texmf-dist/tex/latex/standalone/standalone.cfg)
(/usr/local/texlive/2022/texmf-dist/tex/latex/base/article.cls
Document Class: article 2021/10/04 v1.4n 

In [7]:
Bilevel.print_case_description(model)

┌──────────────┬───────────┬─────┬────────────┬──────────────┬─────────────────┬──────────────┬──────────────┐
│        Model │ Objective │ Gap │ Solve Time │ Time periods │ Energy consumed │ Peak sub max │ Peak sub min │
│              │ kEUR/year │   % │       sec. │              │        MWh/year │   MW on 1 DT │   MW on 1 DT │
├──────────────┼───────────┼─────┼────────────┼──────────────┼─────────────────┼──────────────┼──────────────┤
│ Single Level │  387300.0 │   0 │    0.03476 │            4 │        640900.0 │         2.19 │       0.5098 │
└──────────────┴───────────┴─────┴────────────┴──────────────┴─────────────────┴──────────────┴──────────────┘


3×8 Matrix{Any}:
 "Model"               "Objective"  …   "Peak sub max"   "Peak sub min"
 ""                    "kEUR/year"      "MW on 1 DT"     "MW on 1 DT"
 "Single Level"  387300.0              2.19             0.5098

In [9]:
Bilevel.print_network_results(model)

┌────────────────┬────────────────┬─────────────────┬─────────────────┬─────────────────┬─────────────────┐
│ Nb lines built │ Nb substations │ Nb lines cond 1 │ Nb lines cond 2 │ Nb lines cond 3 │ Nb lines cond 4 │
│              - │              - │               - │               - │               - │               - │
├────────────────┼────────────────┼─────────────────┼─────────────────┼─────────────────┼─────────────────┤
│              2 │              1 │         Int64[] │         Int64[] │          [2, 3] │         Int64[] │
└────────────────┴────────────────┴─────────────────┴─────────────────┴─────────────────┴─────────────────┘


3×6 Matrix{Any}:
  "Nb lines built"   "Nb substations"  …  "Nb lines cond 4"
  "-"                "-"                  "-"
 2                  1                     Int64[]

In [16]:
using JuMP
function print_cost_results(model; latex=false)
    network = model[:network_data]
    DSO_costs = model[:DSO_costs]


    L = get_nb_lines(network)
    K = get_nb_conductors(network)
    lines = network.lines 
    conductors = network.conductors

    # Investements to build new lines in k€
    DSO_fixed_line = value(sum(model[:Alpha][l, k] * lines[l].length * conductors[k].cost for l in 1:L, k in 1:K))

    BASE_POWER = network.pu_basis.base_power
    SUB_COST = DSO_costs.substation
    Ns = get_nb_substations(network)
   
    # Investements to build new substations in k€
    DSO_fixed_sub = value(sum(model[:S_sub_capa][i] * BASE_POWER * SUB_COST for i in 1:Ns))

   
    # Operational costs of losses on the length of the horizon
    DSO_loss = value(model[:DSO_loss_costs]) * DSO_costs.amortization

    # DSO future value: DSO should get back what he must pay per year for the investment + a certain margin
    DSO_future_value = value(model[:DSO_fixed_costs]) * ((1 + DSO_costs.interest_rate)^DSO_costs.amortization) + value(model[:DSO_loss_costs]) * DSO_costs.amortization

    # DSO revenues
    DSO_revenues = value(sum(model[:grid_costs])) * DSO_costs.amortization
    # Other costs
    user_costs = value(sum(model[:user_costs]))
    PV_costs = value(sum(model[:PV_costs]))
    grid_costs = value(sum(model[:grid_costs]))
    energy_costs = value(sum(model[:energy_costs]))
    energy_revenues = value(sum(model[:energy_revenues]))
    if model[:storage]
        stor_costs = value(sum(model[:storage_costs]))
        kpis_header = (["DSO fixed line", "DSO fixed sub", "DSO loss", "DSO future value", "DSO revenues", "User", "PV", "Storage", "Grid connection", "Energy imported", "Energy exported"],
        ["kEUR", "kEUR", "kEUR", "kEUR", "kEUR", "kEUR/year", "kEUR/year", "kEUR/year", "kEUR/year", "kEUR/year", "kEUR/year"])
        kpis = sig_round([DSO_fixed_line DSO_fixed_sub DSO_loss DSO_future_value DSO_revenues user_costs PV_costs stor_costs grid_costs energy_costs energy_revenues])
    end
    kpis_header = (["DSO fixed line", "DSO fixed sub", "DSO loss", "DSO future value", "DSO revenues", "User", "PV", "Grid connection", "Energy imported", "Energy exported"],
        ["kEUR", "kEUR", "kEUR", "kEUR", "kEUR", "kEUR/year", "kEUR/year", "kEUR/year", "kEUR/year", "kEUR/year"])
    kpis = sig_round([DSO_fixed_line DSO_fixed_sub DSO_loss DSO_future_value DSO_revenues user_costs PV_costs grid_costs energy_costs energy_revenues])
    if latex
        pretty_table(kpis, header=kpis_header, backend=Val(:latex))
    else
        pretty_table(kpis, header=kpis_header)
    end
    # returns the table as a Matrix
    return vcat([i[j] for i in kpis_header, j in 1:length(kpis_header[1])], kpis)
end

print_cost_results (generic function with 1 method)

In [12]:
function sig_round(x)
    for (i, a) in enumerate(x)
        if isa(a, AbstractFloat)
            x[i] = round(a, sigdigits=4)
            if abs(a) < 1e-3
                x[i] = 0
            end
        end
    end
    return x
end

sig_round (generic function with 1 method)

In [24]:
print_cost_results(model)

┌────────────────┬───────────────┬──────────┬──────────────────┬──────────────┬───────────┬───────────┬─────────────────┬─────────────────┬─────────────────┐
│ DSO fixed line │ DSO fixed sub │ DSO loss │ DSO future value │ DSO revenues │      User │        PV │ Grid connection │ Energy imported │ Energy exported │
│           kEUR │          kEUR │     kEUR │             kEUR │         kEUR │ kEUR/year │ kEUR/year │       kEUR/year │       kEUR/year │       kEUR/year │
├────────────────┼───────────────┼──────────┼──────────────────┼──────────────┼───────────┼───────────┼─────────────────┼─────────────────┼─────────────────┤
│          3.621 │        2309.0 │    4.378 │          42610.0 │      1.923e7 │  387200.0 │      16.0 │        384600.0 │          2650.0 │             0.0 │
└────────────────┴───────────────┴──────────┴──────────────────┴──────────────┴───────────┴───────────┴─────────────────┴─────────────────┴─────────────────┘


2-element Vector{Float64}:
 214.06698331720432
 106.41348398836253

In [25]:
function print_case_results(model; latex=false)
    DAYS_A_YEAR = 365
    DELTA_T = model[:delta_t]
    NB_PROFILES = model[:nb_sign_days]
   
    MULTIPLIER = DELTA_T/60 * DAYS_A_YEAR / NB_PROFILES
    # PV penetration : p_pv max = peak PV 
    PV_penetration = sum(value.(model[:p_pv_max]))
    # PV energy
    PV_energy = sum(value.(model[:p_pv])) * MULTIPLIER / PV_penetration

    # PV potential
    network = model[:network_data]
    Nu = get_nb_loads(network)
    Ns = get_nb_substations(network)
    T  = model[:time_steps]
    buses = network.buses 
    PV_prod = [(isnothing(buses[Ns + i].PV_installation) ? zeros(Float64, T) : buses[Ns + i].PV_installation.profile.time_serie) for i in 1:Nu]

    PV_potential = sum(PV_prod[i][t] * value(model[:p_pv_max][i]) for i in 1:Nu, t in 1:T) * MULTIPLIER / PV_penetration

    # Energy exchanges with grid
    energy_to_grid = value(sum(model[:p_exp])) * MULTIPLIER
    energy_from_grid = value(sum(model[:p_imp])) * MULTIPLIER

    # Substations capa
    sub_init_capa = [buses[i].S_rating for i in 1:Ns]
    substation1_capacity = sub_init_capa[1]
    substation2_capacity = sub_init_capa[2]
    kpis_header = (["PV penetration", "PV production", "PV potential", "Energy bought from grid", "Energy sold to grid", "Substation1 capacity", "Substation2 capacity"], ["MVA", "MWh/MVA/year", "MWh/MVA/year", "MWh/year", "MWh/year", "MVA", "MVA"])
    kpis = sig_round([PV_penetration PV_energy PV_potential energy_from_grid energy_to_grid substation1_capacity substation2_capacity])
    if latex
        pretty_table(kpis, header=kpis_header, backend=Val(:latex))
    else
        pretty_table(kpis, header=kpis_header)
    end
    # returns the table as a Matrix
    return vcat([i[j] for i in kpis_header, j in 1:length(kpis_header[1])], kpis)
end

print_case_results (generic function with 1 method)

In [26]:
print_case_results(model)

┌────────────────┬───────────────┬──────────────┬─────────────────────────┬─────────────────────┬──────────────────────┬──────────────────────┐
│ PV penetration │ PV production │ PV potential │ Energy bought from grid │ Energy sold to grid │ Substation1 capacity │ Substation2 capacity │
│            MVA │  MWh/MVA/year │ MWh/MVA/year │                MWh/year │            MWh/year │                  MVA │                  MVA │
├────────────────┼───────────────┼──────────────┼─────────────────────────┼─────────────────────┼──────────────────────┼──────────────────────┤
│            0.4 │        4623.0 │       4623.0 │                  8832.0 │                 0.0 │                  0.0 │                2.309 │
└────────────────┴───────────────┴──────────────┴─────────────────────────┴─────────────────────┴──────────────────────┴──────────────────────┘


3×7 Matrix{Any}:
  "PV penetration"      "PV production"  …   "Substation2 capacity"
  "MVA"                 "MWh/MVA/year"       "MVA"
 0.4                4623.0                  2.309

In [34]:
function print_network_results(model; latex=false)
    network = model[:network_data]
    nb_lines_built = 0
    for l in network.lines 
        if l.built 
            nb_lines_built += 1
        end
    end

    Ns = get_nb_substations(network)
    buses = network.buses
    nb_substations = 0
    for i in 1:Ns
        if buses[i].built
            nb_substations += 1
        end
    end
 
    K = get_nb_conductors(network)
    cond_string = ["Nb lines cond "*string(i) for i in 1:K]

    lines_cond = [[i for (i, a) in enumerate(value.(model[:Alpha])[:, j]) if isapprox(a, 1, rtol=1e-2)] for j in 1:K]

    kpis_header = ([["Nb lines built", "Nb substations"];cond_string], [["-", "-"];fill("-", 1:K)])
    kpis = reshape(string.([[nb_lines_built, nb_substations];lines_cond]),1,:)
    if latex
        pretty_table(kpis, header=kpis_header, backend=Val(:latex))
    else
        pretty_table(kpis, header=kpis_header)
    end
    # returns the table as a Matrix
    return vcat([i[j] for i in kpis_header, j in 1:length(kpis_header[1])], kpis)
end


print_network_results (generic function with 1 method)

In [35]:
print_network_results(model)

┌────────────────┬────────────────┬─────────────────┬─────────────────┬─────────────────┬─────────────────┐
│ Nb lines built │ Nb substations │ Nb lines cond 1 │ Nb lines cond 2 │ Nb lines cond 3 │ Nb lines cond 4 │
│              - │              - │               - │               - │               - │               - │
├────────────────┼────────────────┼─────────────────┼─────────────────┼─────────────────┼─────────────────┤
│              2 │              1 │         Int64[] │         Int64[] │          [2, 3] │         Int64[] │
└────────────────┴────────────────┴─────────────────┴─────────────────┴─────────────────┴─────────────────┘


3×6 Matrix{String}:
 "Nb lines built"  "Nb substations"  "Nb lines cond 1"  …  "Nb lines cond 4"
 "-"               "-"               "-"                   "-"
 "2"               "1"               "Int64[]"             "Int64[]"